In [1]:
%pylab inline
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [2]:
import time
import sys
sys.path.insert(0,'../..')
from keras import losses, metrics
from tensorflow.keras import optimizers
import gp2
from gp2 import Runner

2023-04-25 12:59:44.789759: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-04-25 12:59:46.552422: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2023-04-25 12:59:46.684880: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:0f:00.0 name: A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2023-04-25 12:59:46.687465: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 1 with properties: 
pciBusID: 0000:47:00.0 name: A100-SXM4-40GB computeCapability: 8.0
coreClock: 1.41GHz coreCount: 108 deviceMemorySize: 39.59GiB deviceMemoryBandwidth: 1.41TiB/s
2023-04-25 12:59:46.687485: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
2023-04-2

In [3]:
help(gp2.KResUNet2D)

Help on class KResUNet2D in module gp2.gp2.classifiers.k_res_unet2d:

class KResUNet2D(gp2.gp2.classifiers.base_keras_segmentation_classifier.BaseKerasSegmentationClassifier)
 |  KResUNet2D(input_size=(512, 512, 1), filter_num=None, dilation_num=None, n_labels=1, aspp_num_down=16, aspp_num_up=8, activation='PReLU', output_activation='Sigmoid', batch_norm=False, pool=False, unpool=False, name='resunet', optimizer=None, loss=None, metric=None, verbose=False, workingdir='/tmp')
 |  
 |  KResUNet2D
 |  
 |  Method resolution order:
 |      KResUNet2D
 |      gp2.gp2.classifiers.base_keras_segmentation_classifier.BaseKerasSegmentationClassifier
 |      gp2.gp2.classifiers.classifier.Classifier
 |      builtins.object
 |  
 |  Methods defined here:
 |  
 |  __init__(self, input_size=(512, 512, 1), filter_num=None, dilation_num=None, n_labels=1, aspp_num_down=16, aspp_num_up=8, activation='PReLU', output_activation='Sigmoid', batch_norm=False, pool=False, unpool=False, name='resunet', optimiz

In [11]:
R = Runner(verbose=True, 
           classifier='kresunet2d',
           filter_num=[16, 32, 64, 128, 256], 
           dilation_num=None,
           aspp_num_down=16, 
           aspp_num_up=8, 
           activation='ReLU', 
           output_activation='Sigmoid', 
           batch_norm=False, 
           pool=True, 
           unpool=True,
           optimizer=None, 
           loss=None, 
           metric=None)

Received dilation rates: [1, 3, 15, 31]
Received dilation rates are not defined on a per downsampling level basis.
Automated determinations are applied with the following details:
	depth-0, dilation_rate = [1, 3, 15, 31]
	depth-1, dilation_rate = [1, 3, 15, 31]
	depth-2, dilation_rate = [1, 3, 15]
	depth-3, dilation_rate = [1]
	depth-4, dilation_rate = [1]
*** GP2  KResUNet2D ***
Working directory: /tmp/tmpb1y1ry5jGP2
Verbose mode active!
{'verbose': True, 'workingdir': '/tmp/tmpb1y1ry5jGP2', 'input_size': (512, 512, 1), 'filter_num': [16, 32, 64, 128, 256], 'dilation_num': [1, 3, 15, 31], 'n_labels': 1, 'aspp_num_down': 16, 'aspp_num_up': 8, 'activation': 'ReLU', 'output_activation': 'Sigmoid', 'batch_norm': False, 'pool': True, 'unpool': True, 'name': 'resunet', 'optimizer': <tensorflow.python.keras.optimizer_v2.adam.Adam object at 0x7f80519ff820>, 'loss': <function binary_crossentropy at 0x7f8d08d6db80>, 'metric': [<function binary_accuracy at 0x7f8d08c18af0>], 'model': <tensorflow.

In [5]:
# load our larger toy dataset (10k images and masks)
images = np.load('/hpcstor6/scratch01/r/ryan.zurrin001/GP2TOYEXAMPLE_LARGE/images.npy')
masks = np.load('/hpcstor6/scratch01/r/ryan.zurrin001/GP2TOYEXAMPLE_LARGE/masks.npy')

In [6]:
images = images[:5000]
masks = masks[:5000]

In [7]:
images.shape

(5000, 512, 512, 1)

In [8]:
masks.shape

(5000, 512, 512, 1)

In [9]:
weights = {
    'A': 0.5,
    'A_train': 0.1,
    'A_val': 0.3,
    'A_test': 0.6,
    'B': 0.3,
    'B_train': 0.7,
    'B_val': 0.1,
    'B_test': 0.2,
    'Z': 0.2
}

In [12]:
R.run(images=images, masks=masks, weights=weights, runs=3)

Weights OK!
******
Loop 0
Epoch 1/100


2023-04-25 13:04:14.103866: W tensorflow/core/common_runtime/bfc_allocator.cc:271] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.30GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2023-04-25 13:04:14.384087: W tensorflow/core/common_runtime/bfc_allocator.cc:271] Allocator (GPU_0_bfc) ran out of memory trying to allocate 2.53GiB with freed_by_count=0. The caller indicates that this is not a failure, but may mean that there could be performance gains if more memory were available.
2023-04-25 13:04:24.640025: W tensorflow/core/common_runtime/bfc_allocator.cc:456] Allocator (GPU_0_bfc) ran out of memory trying to allocate 677.58MiB (rounded to 710494208)requested by op resunet_model/resunet_up3_resblock_stack3_1/Conv2D/SpaceToBatchND
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Cu

2023-04-25 13:04:34.655355: W tensorflow/core/common_runtime/bfc_allocator.cc:456] Allocator (GPU_0_bfc) ran out of memory trying to allocate 514.00MiB (rounded to 538970112)requested by op resunet_model/resunet_up3_resblock_stack1_1/Conv2D
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-04-25 13:04:34.655500: I tensorflow/core/common_runtime/bfc_allocator.cc:991] BFCAllocator dump for GPU_0_bfc
2023-04-25 13:04:34.655510: I tensorflow/core/common_runtime/bfc_allocator.cc:998] Bin (256): 	Total Chunks: 677, Chunks in use: 668. 169.2KiB allocated for chunks. 167.0KiB in use in bin. 51.3KiB client-requested in use in bin.
2023-04-25 13:04:34.655516: I tensorflow/core/common_runtime/bfc_allocator.cc:998] Bin (512): 	Total Chunks: 65, Chunks in use: 52. 37.2KiB allocated for chunks. 30.2KiB in use in bin. 27.0KiB client-requested

2023-04-25 13:04:44.672823: W tensorflow/core/common_runtime/bfc_allocator.cc:456] Allocator (GPU_0_bfc) ran out of memory trying to allocate 538.33MiB (rounded to 564480000)requested by op resunet_model/resunet_up3_resblock_stack2_1/Conv2D
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2023-04-25 13:04:44.672978: I tensorflow/core/common_runtime/bfc_allocator.cc:991] BFCAllocator dump for GPU_0_bfc
2023-04-25 13:04:44.672988: I tensorflow/core/common_runtime/bfc_allocator.cc:998] Bin (256): 	Total Chunks: 677, Chunks in use: 668. 169.2KiB allocated for chunks. 167.0KiB in use in bin. 51.3KiB client-requested in use in bin.
2023-04-25 13:04:44.672995: I tensorflow/core/common_runtime/bfc_allocator.cc:998] Bin (512): 	Total Chunks: 65, Chunks in use: 52. 37.2KiB allocated for chunks. 30.2KiB in use in bin. 27.0KiB client-requested

ResourceExhaustedError:  OOM when allocating tensor with shape[61504,19,19,16] and type half on /job:localhost/replica:0/task:0/device:GPU:0 by allocator GPU_0_bfc
	 [[node resunet_model/resunet_up3_resblock_stack3_1/Conv2D/SpaceToBatchND (defined at home/ryan.zurrin001/Projects/CS410-Omama/_EXPERIMENTS/GP2/../../gp2/gp2/classifiers/base_keras_segmentation_classifier.py:45) ]]
Hint: If you want to see a list of allocated tensors when OOM happens, add report_tensor_allocations_upon_oom to RunOptions for current allocation info.
 [Op:__inference_train_function_35821]

Function call stack:
train_function


In [ ]:
R.classifier_scores

In [ ]:
R.discriminator_scores

In [ ]:
R.plot()